In [1]:
import numpy as np 
import pandas as pd 
import json
from tqdm.notebook import tqdm
import xmltodict

import os
for dirname, _, filenames in os.walk('../data/drugs/drugbank_5.1.10/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../data/drugs/drugbank_5.1.10/3D structures.sdf
../data/drugs/drugbank_5.1.10/drug links.csv
../data/drugs/drugbank_5.1.10/drug sequences.fasta
../data/drugs/drugbank_5.1.10/drugbank vocabulary.csv
../data/drugs/drugbank_5.1.10/full database.7z
../data/drugs/drugbank_5.1.10/full database.xml
../data/drugs/drugbank_5.1.10/protein_identifiers.csv
../data/drugs/drugbank_5.1.10/structure links.csv
../data/drugs/drugbank_5.1.10/structures.sdf
../data/drugs/drugbank_5.1.10/target_sequences_gene.fasta
../data/drugs/drugbank_5.1.10/target_sequences_protein.fasta
../data/drugs/drugbank_5.1.10/uniprot links.csv


In [2]:
with open('../data/drugs/drugbank_5.1.10/full database.xml', 'r',encoding='utf-8') as xml_file:
    print("Deserializing XML...")
    dat = xml_file.read()
    data_dict = xmltodict.parse(dat)
    print("Done.")
    
dat = data_dict['drugbank']["drug"]
print(len(dat),"objects found.")

# print("Serializing to JSON...")
# with open("drugbank_full.json", 'w') as fp:
#         json.dump(dat, fp, sort_keys=True, indent=4)
# print("Done.")

Deserializing XML...
Done.
15235 objects found.


# Process Drug Nodes

In [3]:
#node table
# drug ids

drug_dicts = []

for obj in tqdm(dat):
    
    # name
    objname = obj['name']
    filetype = obj['@type']
    
    # case
    try:
        cas = obj["cas-number"]
    except:
        cas = ""
    
    # atc
    try:
        atc = obj["atc-codes"]["atc-code"]
        if type(atc) == dict:
            atc = atc["@code"]
        if type(atc) == list:
            atc = atc[0]["@code"]
    except:
        if atc is not None:
            atc = obj["atc-codes"]
        else:
            atc = ""
    #rxcui        
    try:        
        ext_ids = obj['external-identifiers']['external-identifier']
        for item in ext_ids:
            if(item['resource'] == 'RxCUI'):
                rxcui = item['identifier']
    except:
        rxcui=""

    # drugbank-id
    try:
        k,v = obj['drugbank-id'][0].items()
        fileid = v[1]
    except:
        fileid = obj['drugbank-id']['#text']
        
    #Unii
    try:
        unii = obj['unii']
    except:
        unii = ""
    
    drug_row = {
        'name': objname,
        'drugbank-id': fileid,
        'type':filetype,
        'atc':atc,
        'cas':cas,
        'rxcui':rxcui,
        'unii':unii
    }
    drug_dicts.append(drug_row)
    
            
print("Done.")

  0%|          | 0/15235 [00:00<?, ?it/s]

Done.


In [5]:
dat[0]

{'@type': 'biotech',
 '@created': '2005-06-13',
 '@updated': '2023-01-03',
 'drugbank-id': [{'@primary': 'true', '#text': 'DB00001'},
  'BTD00024',
  'BIOD00024'],
 'name': 'Lepirudin',
 'description': 'Lepirudin is a recombinant hirudin formed by 65 amino acids that acts as a highly specific and direct thrombin inhibitor.[L41539,L41569] Natural hirudin is an endogenous anticoagulant found in _Hirudo medicinalis_ leeches.[L41539] Lepirudin is produced in yeast cells and is identical to natural hirudin except for the absence of sulfate on the tyrosine residue at position 63 and the substitution of leucine for isoleucine at position 1 (N-terminal end).[A246609] \r\n\r\nLepirudin is used as an anticoagulant in patients with heparin-induced thrombocytopenia (HIT), an immune reaction associated with a high risk of thromboembolic complications.[A3, L41539] HIT is caused by the expression of immunoglobulin G (IgG) antibodies that bind to the complex formed by heparin and platelet factor 4. Th

In [4]:
drugs_df = pd.DataFrame(drug_dicts)
drugs_df

,name,drugbank-id,type,atc,cas,rxcui,unii
0,Lepirudin,DB00001,biotech,B01AE02,138068-37-8,237057,Y43GF64R34
1,Cetuximab,DB00002,biotech,L01FE01,205923-56-4,318341,PQX0D8J21J
2,Dornase alfa,DB00003,biotech,R05CB13,143831-71-4,337623,953A26OA1Y
3,Denileukin diftitox,DB00004,biotech,L01XX29,173146-27-5,214470,25E79B5CTM
4,Etanercept,DB00005,biotech,L04AB01,185243-69-0,214555,OP401G7OJC
...,...,...,...,...,...,...,...
15230,AUM-601,DB17382,small molecule,None,None,,None
15231,FN-1501,DB17383,small molecule,,1429515-59-2,,6MC966B505
15232,Tinengotinib,DB17384,small molecule,None,2230490-29-4,,WZ9TJ0L9Y8
15233,Lipotecan,DB17385,small molecule,,1432468-79-5,,D47234N30N


In [5]:
drugs_syn_df = pd.read_csv(r"../data/drugs/drugbank_5.1.10/drugbank vocabulary.csv")
drugs_syn_df.fillna("",inplace=True)
drugs_syn_df = drugs_syn_df[['DrugBank ID', 'Accession Numbers', 'Synonyms', 'Standard InChI Key']]
drugs_syn_df.columns = ['drugbank-id', 'drugbank-id-2', 'synonyms', 'InChI']
drugs_syn_df

,drugbank-id,drugbank-id-2,synonyms,InChI
0,DB00001,BIOD00024 | BTD00024,"[Leu1, Thr2]-63-desulfohirudin | Desulfatohiru...",
1,DB00002,BIOD00071 | BTD00071,Cetuximab | Cétuximab | Cetuximabum,
2,DB00003,BIOD00001 | BTD00001,Deoxyribonuclease (human clone 18-1 protein mo...,
3,DB00004,BIOD00084 | BTD00084,Denileukin | Denileukin diftitox | Interleukin...,
4,DB00005,BIOD00052 | BTD00052,Etanercept | etanercept-szzs | etanercept-ykro...,
...,...,...,...,...
15230,DB17382,,"(R,E)-3-(5-(2-(2,5-difluorophenyl)pyrrolidin-1...",
15231,DB17383,,"4-((7h-pyrrolo (2,3-d)pyrimidin-4-yl)amino)-n-...",VXLAKHWYGRKCGI-UHFFFAOYSA-N
15232,DB17384,,"4-(5-(2-Chlorophenyl)-3-Methyl-2,10-Dihydropyr...",DQFCVOOFMXEPOC-UHFFFAOYSA-N
15233,DB17385,,"Lipothecan free base | Propanoic acid, 2-(((2,...",JCCCLGDYMMTBPM-HXDHBHDHSA-N


In [6]:
drugs_df = pd.merge(left=drugs_df, right=drugs_syn_df, on='drugbank-id')
drugs_df.fillna("", inplace=True)
drugs_df

,name,drugbank-id,type,atc,cas,rxcui,unii,drugbank-id-2,synonyms,InChI
0,Lepirudin,DB00001,biotech,B01AE02,138068-37-8,237057,Y43GF64R34,BIOD00024 | BTD00024,"[Leu1, Thr2]-63-desulfohirudin | Desulfatohiru...",
1,Cetuximab,DB00002,biotech,L01FE01,205923-56-4,318341,PQX0D8J21J,BIOD00071 | BTD00071,Cetuximab | Cétuximab | Cetuximabum,
2,Dornase alfa,DB00003,biotech,R05CB13,143831-71-4,337623,953A26OA1Y,BIOD00001 | BTD00001,Deoxyribonuclease (human clone 18-1 protein mo...,
3,Denileukin diftitox,DB00004,biotech,L01XX29,173146-27-5,214470,25E79B5CTM,BIOD00084 | BTD00084,Denileukin | Denileukin diftitox | Interleukin...,
4,Etanercept,DB00005,biotech,L04AB01,185243-69-0,214555,OP401G7OJC,BIOD00052 | BTD00052,Etanercept | etanercept-szzs | etanercept-ykro...,
...,...,...,...,...,...,...,...,...,...,...
15230,AUM-601,DB17382,small molecule,,,,,,"(R,E)-3-(5-(2-(2,5-difluorophenyl)pyrrolidin-1...",
15231,FN-1501,DB17383,small molecule,,1429515-59-2,,6MC966B505,,"4-((7h-pyrrolo (2,3-d)pyrimidin-4-yl)amino)-n-...",VXLAKHWYGRKCGI-UHFFFAOYSA-N
15232,Tinengotinib,DB17384,small molecule,,2230490-29-4,,WZ9TJ0L9Y8,,"4-(5-(2-Chlorophenyl)-3-Methyl-2,10-Dihydropyr...",DQFCVOOFMXEPOC-UHFFFAOYSA-N
15233,Lipotecan,DB17385,small molecule,,1432468-79-5,,D47234N30N,,"Lipothecan free base | Propanoic acid, 2-(((2,...",JCCCLGDYMMTBPM-HXDHBHDHSA-N


In [7]:
drugs_df.to_csv(r"./temp/drug_nodes_identifiers.tsv", sep="\t", index=False)
drugs_df.to_excel(r"./temp/drug_nodes_identifiers.xlsx", index=False)

### TODO: Process Drug structures

In [9]:
from rdkit.Chem import PandasTools
SDFFile = "../data/drugs/drugbank_5.1.10/structures.sdf"
struct_df = PandasTools.LoadSDF(SDFFile)
struct_df

[21:20:35] Explicit valence for atom # 13 Cl, 5, is greater than permitted
[21:20:35] ERROR: Could not sanitize molecule ending on line 298551
[21:20:35] ERROR: Explicit valence for atom # 13 Cl, 5, is greater than permitted
[21:20:35] Explicit valence for atom # 19 O, 3, is greater than permitted
[21:20:35] ERROR: Could not sanitize molecule ending on line 412786
[21:20:35] ERROR: Explicit valence for atom # 19 O, 3, is greater than permitted
[21:20:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[21:20:35] ERROR: Could not sanitize molecule ending on line 540739
[21:20:35] ERROR: Explicit valence for atom # 1 N, 4, is greater than permitted
[21:20:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[21:20:35] ERROR: Could not sanitize molecule ending on line 598037
[21:20:35] ERROR: Explicit valence for atom # 1 N, 4, is greater than permitted
[21:20:35] Explicit valence for atom # 12 N, 4, is greater than permitted
[21:20:35] ERROR: Could not sanitize

,DATABASE_ID,DATABASE_NAME,SMILES,INCHI_IDENTIFIER,INCHI_KEY,FORMULA,MOLECULAR_WEIGHT,EXACT_MASS,JCHEM_ACCEPTOR_COUNT,JCHEM_ATOM_COUNT,...,DRUGBANK_ID,SECONDARY_ACCESSION_NUMBERS,DRUG_GROUPS,GENERIC_NAME,SYNONYMS,PRODUCTS,INTERNATIONAL_BRANDS,ID,ROMol,SALTS
0,DB00006,drugbank,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,OIRCOABEOLEUMC-GEJPAHFPSA-N,C98H138N24O33,2180.2853,2178.985813062,37,293,...,DB00006,BTD00076; EXPT03302; BIOD00076; DB02351,approved; investigational,Bivalirudin,Bivalirudin; Bivalirudina; Bivalirudinum,Angiomax; Angiomax RTU; Angiox; Bivalirudin; B...,Angiox; Hirulog,,<rdkit.Chem.rdchem.Mol object at 0x000001AD092...,NaN
1,DB00007,drugbank,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,GFIJNRVAKGFPGQ-LIJARHBVSA-N,C59H84N16O12,1209.3983,1208.645462232,16,171,...,DB00007,BTD00009; BIOD00009,approved; investigational,Leuprolide,Leuprorelin; Leuprorelina; Leuproreline; Leupr...,Camcevi; Eligard; Fensolvi; Leuprolide Acetate...,Camcevi; Leuplin; LeuProMaxx; Memryte; Prostap...,,<rdkit.Chem.rdchem.Mol object at 0x000001AD093...,Leuprolide acetate; Leuprolide mesylate
2,DB00014,drugbank,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,BLCLNMBMMGCOAS-URPVMXJPSA-N,C59H84N18O14,1269.4105,1268.641439486,18,175,...,DB00014,BTD00113; BIOD00113,approved,Goserelin,Goserelin; Goserelina,Zoladex; Zoladex LA,NaN,[NO NAME],<rdkit.Chem.rdchem.Mol object at 0x000001AD093...,Goserelin acetate
3,DB00027,drugbank,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,NDAYQJDHGXTBJL-MWWSRJDJSA-N,C96H135N19O16,1811.253,1810.033419343,16,266,...,DB00027,BTD00036; BIOD00036,approved,Gramicidin D,Bacillus brevis gramicidin D; Gramicidin; Gram...,Antibiotic Cream; Antibiotic Cream Plus Pain R...,Sofradex,,<rdkit.Chem.rdchem.Mol object at 0x000001AD093...,NaN
4,DB00035,drugbank,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,NFLWUMRGJYTJIN-PNIOQBSNSA-N,C46H64N14O12S2,1069.22,1068.426955905,15,138,...,DB00035,BTD00112; BTD00061; BIOD00112; BIOD00061,approved,Desmopressin,1-(3-mercaptopropionic acid)-8-D-arginine-vaso...,Apo-desmopressin; Bipazen; DDAVP Rhinal Tube; ...,Adiuretin; DesmoMelt,,<rdkit.Chem.rdchem.Mol object at 0x000001AD093...,Desmopressin acetate; Desmopressin acetate anh...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11581,DB17379,drugbank,CC(C)C1=C(O)C(O)=C(C=O)C2=C(O)C(=C(C)C=C12)C1=...,InChI=1S/C30H30O8/c1-11(2)19-15-7-13(5)21(27(3...,QBKSWRVVCFFDOT-UHFFFAOYSA-N,C30H30O8,518.5544,518.194067936,8,68,...,DB17379,NaN,investigational,(-)-Gossypol,"(r)-2,2'-bis(8-formyl-1,6,7-trihydroxy-5-isopr...",NaN,NaN,,<rdkit.Chem.rdchem.Mol object at 0x000001AD1FE...,NaN
11582,DB17383,drugbank,CN1CCN(CC2=CC=C(NC(=O)C3=NNC=C3NC3=C4C=CNC4=NC...,InChI=1S/C22H25N9O/c1-30-8-10-31(11-9-30)13-15...,VXLAKHWYGRKCGI-UHFFFAOYSA-N,C22H25N9O,431.504,431.218206461,NaN,NaN,...,DB17383,NaN,investigational,FN-1501,"4-((7h-pyrrolo (2,3-d)pyrimidin-4-yl)amino)-n-...",NaN,NaN,,<rdkit.Chem.rdchem.Mol object at 0x000001AD1FE...,NaN
11583,DB17384,drugbank,CC1=C2N=C(C3=CC=CC=C3Cl)C3=C(NC2=NN1)C=C(N=C3)...,InChI=1S/C20H19ClN6O/c1-12-18-20(26-25-12)23-1...,DQFCVOOFMXEPOC-UHFFFAOYSA-N,C20H19ClN6O,394.86,394.130887,NaN,NaN,...,DB17384,NaN,investigational,Tinengotinib,"4-(5-(2-Chlorophenyl)-3-Methyl-2,10-Dihydropyr...",NaN,NaN,,<rdkit.Chem.rdchem.Mol object at 0x000001AD1FE...,NaN
11584,DB17385,drugbank,CC[C@@]1(OC(=O)C(C)ON=C2C3=C(C4=C2C=C(C=C4[N+]...,InChI=1S/C39H30N8O15/c1-5-39(26-13-30-34-18(14...,JCCCLGDYMMTBPM-HXDHBHDHSA-N,C39H30N8O15,850.71,850.183062298,NaN,NaN,...,DB17385,NaN,investigational,Lipotecan,"Lipothecan free base; Propanoic acid, 2-(((2,4...",NaN,NaN,,<rdkit.Chem.rdchem.Mol object at 0x000001AD1FE...,Lipotecan hydrochloride


In [10]:
struct_df = struct_df[['DATABASE_ID', 'SMILES']]
struct_df.dropna(inplace=True)
struct_df

C:\Users\Rangan\AppData\Local\Temp\ipykernel_12464\99577163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  struct_df.dropna(inplace=True)


,DATABASE_ID,SMILES
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
4,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
...,...,...
11581,DB17379,CC(C)C1=C(O)C(O)=C(C=O)C2=C(O)C(=C(C)C=C12)C1=...
11582,DB17383,CN1CCN(CC2=CC=C(NC(=O)C3=NNC=C3NC3=C4C=CNC4=NC...
11583,DB17384,CC1=C2N=C(C3=CC=CC=C3Cl)C3=C(NC2=NN1)C=C(N=C3)...
11584,DB17385,CC[C@@]1(OC(=O)C(C)ON=C2C3=C(C4=C2C=C(C=C4[N+]...


# Targets & MOA edges

In [11]:
target_dicts = []

for i, obj in enumerate(dat):
    # get drug id
    drug_id = None
    action = None
    drug_type = obj['@type']
    
    try:
        k,v = obj['drugbank-id'][0].items()
        drug_id = v[1]
    except:
        drug_id = obj['drugbank-id']['#text']
        
    # get target uniprotkb id
    uniprotkb_id = None
    uniprotac_id = None
    gene_name = None

    if obj['targets'] != None:
        # if a single target is stored as a dict
        if type(obj['targets']['target'])==dict:
            try:
                uniprotkb_id = obj['targets']['target']['polypeptide']['external-identifiers']['external-identifier'][-2]['identifier']
                uniprotac_id = obj['targets']['target']['polypeptide']['external-identifiers']['external-identifier'][-1]['identifier']
                gene_name = obj['targets']['target']['polypeptide']['gene-name']
                action = obj['targets']['target']['actions']['action']
                target_dicts.append([drug_id, drug_type, uniprotkb_id, uniprotac_id, gene_name, action])
            except:
                pass

        else:
            tgts = len(obj['targets']['target'])
            for idx in range(tgts):
                if 'polypeptide' in obj['targets']['target'][idx].keys():
                    if(type(obj['targets']['target'][idx]['polypeptide'])==dict):
                        try:
                            uniprotkb_id = obj['targets']['target'][idx]['polypeptide']['external-identifiers']['external-identifier'][-2]['identifier']
                            uniprotac_id = obj['targets']['target'][idx]['polypeptide']['external-identifiers']['external-identifier'][-1]['identifier']
                            gene_name = obj['targets']['target'][idx]['polypeptide']['gene-name']
                            action = obj['targets']['target'][idx]['actions']['action']
                            target_dicts.append([drug_id, drug_type, uniprotkb_id, uniprotac_id, gene_name, action])
                        except:
                            pass
                    else:
                        ppds = len(obj['targets']['target'][idx]['polypeptide'])
                        for idx_ in range(ppds):
                            try:
                                #print(obj['targets']['target'][idx]['polypeptide'][idx_]['gene-name'])
                                gene_name = obj['targets']['target'][idx]['polypeptide'][idx_]['gene-name']
                                uniprotkb_id = obj['targets']['target'][idx]['polypeptide'][idx_]['external-identifiers']['external-identifier'][-2]['identifier']
                                uniprotac_id = obj['targets']['target'][idx]['polypeptide'][idx_]['external-identifiers']['external-identifier'][-1]['identifier']
                                action = obj['targets']['target'][idx]['actions']['action']
                                target_dicts.append([drug_id, drug_type, uniprotkb_id, uniprotac_id, gene_name, action])
                            except:
                                continue
    else:
        target_dicts.append([drug_id, drug_type, uniprotkb_id, uniprotac_id, gene_name, action])


targets_df = pd.DataFrame(data=target_dicts, columns=["drugbank-id", "drug-type" ,"uniprotkb-id", "uniprotac-id","gene-name","action"])

In [12]:
targets_df_expl = []

for idx, row in targets_df.iterrows():
    moa = row.action
    row_l = row.to_list()
    row_l = row_l[:-1]
    if type(moa) == list:
        for a in moa:
            targets_df_expl.append(row_l + [a])
    else:
        targets_df_expl.append(row.to_list())

targets_df_expl = pd.DataFrame(data=targets_df_expl, columns=["drugbank-id", "drug-type" ,"uniprotkb-id", "uniprotac-id","gene-name","action"])    


In [13]:
targets_df_expl.to_excel(r"./temp/drug_target_edges.xlsx", index=False)
targets_df_expl.dropna().to_excel(r"./temp/drug_target_edges_nonulls.xlsx", index=False)

## Build drug-target-moa dataset

In [14]:
smiles_df = struct_df
smiles_df.columns = ["drugbank-id","structure"]
smiles_df["type"] = ['SMILES']*len(smiles_df)
smiles_df

# drug_target_moa_df = pd.merge()

C:\Users\Rangan\AppData\Local\Temp\ipykernel_12464\3704952450.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smiles_df["type"] = ['SMILES']*len(smiles_df)


,drugbank-id,structure,type
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,SMILES
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,SMILES
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,SMILES
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,SMILES
4,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,SMILES
...,...,...,...
11581,DB17379,CC(C)C1=C(O)C(O)=C(C=O)C2=C(O)C(=C(C)C=C12)C1=...,SMILES
11582,DB17383,CN1CCN(CC2=CC=C(NC(=O)C3=NNC=C3NC3=C4C=CNC4=NC...,SMILES
11583,DB17384,CC1=C2N=C(C3=CC=CC=C3Cl)C3=C(NC2=NN1)C=C(N=C3)...,SMILES
11584,DB17385,CC[C@@]1(OC(=O)C(C)ON=C2C3=C(C4=C2C=C(C=C4[N+]...,SMILES


In [15]:
targets_df_expl

,drugbank-id,drug-type,uniprotkb-id,uniprotac-id,gene-name,action
0,DB00001,biotech,P00734,THRB_HUMAN,F2,inhibitor
1,DB00002,biotech,P00533,EGFR_HUMAN,EGFR,binder
2,DB00002,biotech,O75015,FCG3B_HUMAN,FCGR3B,binder
3,DB00002,biotech,P02745,C1QA_HUMAN,C1QA,binder
4,DB00002,biotech,P02746,C1QB_HUMAN,C1QB,binder
...,...,...,...,...,...,...
19107,DB17382,small molecule,None,None,None,None
19108,DB17383,small molecule,None,None,None,None
19109,DB17384,small molecule,None,None,None,None
19110,DB17385,small molecule,None,None,None,None


In [16]:
len(set(targets_df_expl['drugbank-id'].to_list()))

10012

In [18]:
from Bio import SeqIO

drug_seq = []
for seq_record in SeqIO.parse("../data/drugs/drugbank_5.1.10/drug sequences.fasta", "fasta"):
    drug_seq.append(
        {
        	'drugbank-id':seq_record.id.split('|')[1],	
            'structure': str(seq_record.seq),
            'type':'Protein'
        }
    )


drug_seq_df = pd.DataFrame(drug_seq)
drug_seq_df

,drugbank-id,structure,type
0,DB00001,LTYTDCTESGQNLCLCEGSNVCGQGNKCILGSDGEKNQCVTGEGTP...,Protein
1,DB00002,QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLE...,Protein
2,DB00002,DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRL...,Protein
3,DB00003,LKIAAFNIQTFGETKMSNATLVSYIVQILSRYDIALVQEVRDSHLT...,Protein
4,DB00004,MGADDVVDSSKSFVMENFSSYHGTKPGYVDSIQKGIQKPKSGTQGN...,Protein
...,...,...,...
433,DB16701,MEHKEVVLLLLLFLKSGQGEPLDDYVNTQGASLFSVTKKQLGAGSI...,Protein
434,DB16755,QITLKESGPTLVKPTQTLTLTCTFSGFSLSISGVGVGWLRQPPGKA...,Protein
435,DB16755,QSALTQPASVSGSPGQSITISCTATSSDVGDYNYVSWYQQHPGKAP...,Protein
436,DB17012,MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYM...,Protein


In [19]:
all_struct_df = pd.concat([smiles_df, drug_seq_df])
all_struct_df.to_excel('temp/all_struct.xlsx', index=False)

In [20]:
prot_seq = []
for seq_record in SeqIO.parse("../data/drugs/drugbank_5.1.10/target_sequences_protein.fasta", "fasta"):
    prot_seq.append(
        {
        	'uniprotkb-id':seq_record.id.split('|')[1],	
            'aa-seq': str(seq_record.seq),
        }
    )

prot_seq_df = pd.DataFrame(prot_seq)
prot_seq_df

,uniprotkb-id,aa-seq
0,P45059,MVKFNSSRKSGKSKKTIRKLTAPETVKQNKPQKVFEKCFMRGRYML...
1,P19113,MMEPEEYRERGREMVDYICQYLSTVRERRVTPDVQPGYLRAQLPES...
2,Q9UI32,MRSMKALQKALSRAGSHCGRGGWGHPSRSPLLGGGVRHHLSEAAAQ...
3,P00488,MSETSRTAFGGRRAVPPNNSNAAEDDLPTVELQGVVPRGVNLQEFL...
4,P35228,MACPWKFLFKTKFHQYAMNGEKDINNNVEKAPCATSSPVTQDDLQY...
...,...,...
5341,P20963,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...
5342,Q8IAS0,MKRISPLNTLFYLSLFFSYTFKGLKCTRIYKIGTKALPCSECHDVF...
5343,W7JWW5,MKRISPLNTLFYLSLFFSYTFKGLKCTRIYKIGTKALPCSECHDVF...
5344,P03420,MELLILKANAITTILTAVTFCFASGQNITEEFYQSTCSAVSKGYLS...


In [21]:
df1 = pd.merge(left=targets_df_expl, right=all_struct_df, on='drugbank-id')

In [22]:
df2 = pd.merge(left=df1, right=prot_seq_df, on='uniprotkb-id')
df2

,drugbank-id,drug-type,uniprotkb-id,uniprotac-id,gene-name,action,structure,type,aa-seq
0,DB00001,biotech,P00734,THRB_HUMAN,F2,inhibitor,LTYTDCTESGQNLCLCEGSNVCGQGNKCILGSDGEKNQCVTGEGTP...,Protein,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
1,DB00006,small molecule,P00734,THRB_HUMAN,F2,inhibitor,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,SMILES,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
2,DB00170,small molecule,P00734,THRB_HUMAN,F2,activator,CC1=CC(=O)C2=CC=CC=C2C1=O,SMILES,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
3,DB00278,small molecule,P00734,THRB_HUMAN,F2,inhibitor,C[C@@H]1CCN([C@H](C1)C(O)=O)C(=O)[C@H](CCCNC(N...,SMILES,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
4,DB01123,small molecule,P00734,THRB_HUMAN,F2,other/unknown,NC1=CC2=NC3=C(C=CC(N)=C3)C=C2C=C1,SMILES,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
...,...,...,...,...,...,...,...,...,...
20956,DB16655,biotech,Q02223,TNR17_HUMAN,TNFRSF17,antibody,QTVVTQEPSLTVSPGGTVTLTCRSSTGAVTTSNYANWVQQKPGQAP...,Protein,MLQMAGQCSQNEYFDSLLHACIPCQLRCSSNTPPLTCQRYCNASVT...
20957,DB17096,small molecule,Q8IAS0,PLM10_PLAF7,PMX,inhibitor,C[C@H]1C[C@H](CCO1)N1C(=O)C[C@](C)(NC1=N)C1=C(...,SMILES,MKRISPLNTLFYLSLFFSYTFKGLKCTRIYKIGTKALPCSECHDVF...
20958,DB17096,small molecule,Q8IAS0,PLM10_PLAF7,PMX,binder,C[C@H]1C[C@H](CCO1)N1C(=O)C[C@](C)(NC1=N)C1=C(...,SMILES,MKRISPLNTLFYLSLFFSYTFKGLKCTRIYKIGTKALPCSECHDVF...
20959,DB17096,small molecule,W7JWW5,PLM10_PLAFO,PMX,inhibitor,C[C@H]1C[C@H](CCO1)N1C(=O)C[C@](C)(NC1=N)C1=C(...,SMILES,MKRISPLNTLFYLSLFFSYTFKGLKCTRIYKIGTKALPCSECHDVF...


In [23]:
df2.drop_duplicates(inplace=True)
df2

,drugbank-id,drug-type,uniprotkb-id,uniprotac-id,gene-name,action,structure,type,aa-seq
0,DB00001,biotech,P00734,THRB_HUMAN,F2,inhibitor,LTYTDCTESGQNLCLCEGSNVCGQGNKCILGSDGEKNQCVTGEGTP...,Protein,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
1,DB00006,small molecule,P00734,THRB_HUMAN,F2,inhibitor,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,SMILES,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
2,DB00170,small molecule,P00734,THRB_HUMAN,F2,activator,CC1=CC(=O)C2=CC=CC=C2C1=O,SMILES,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
3,DB00278,small molecule,P00734,THRB_HUMAN,F2,inhibitor,C[C@@H]1CCN([C@H](C1)C(O)=O)C(=O)[C@H](CCCNC(N...,SMILES,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
4,DB01123,small molecule,P00734,THRB_HUMAN,F2,other/unknown,NC1=CC2=NC3=C(C=CC(N)=C3)C=C2C=C1,SMILES,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
...,...,...,...,...,...,...,...,...,...
20956,DB16655,biotech,Q02223,TNR17_HUMAN,TNFRSF17,antibody,QTVVTQEPSLTVSPGGTVTLTCRSSTGAVTTSNYANWVQQKPGQAP...,Protein,MLQMAGQCSQNEYFDSLLHACIPCQLRCSSNTPPLTCQRYCNASVT...
20957,DB17096,small molecule,Q8IAS0,PLM10_PLAF7,PMX,inhibitor,C[C@H]1C[C@H](CCO1)N1C(=O)C[C@](C)(NC1=N)C1=C(...,SMILES,MKRISPLNTLFYLSLFFSYTFKGLKCTRIYKIGTKALPCSECHDVF...
20958,DB17096,small molecule,Q8IAS0,PLM10_PLAF7,PMX,binder,C[C@H]1C[C@H](CCO1)N1C(=O)C[C@](C)(NC1=N)C1=C(...,SMILES,MKRISPLNTLFYLSLFFSYTFKGLKCTRIYKIGTKALPCSECHDVF...
20959,DB17096,small molecule,W7JWW5,PLM10_PLAFO,PMX,inhibitor,C[C@H]1C[C@H](CCO1)N1C(=O)C[C@](C)(NC1=N)C1=C(...,SMILES,MKRISPLNTLFYLSLFFSYTFKGLKCTRIYKIGTKALPCSECHDVF...


In [24]:
df2.to_excel("temp/drug-target-moa.xlsx", index=False)

In [25]:
#END

# Drug Drug interactions

In [26]:
intr_dicts = []

for obj in tqdm(dat):
    try:
        k,v = obj['drugbank-id'][0].items()
        drug_id = v[1]
    except:
        drug_id = obj['drugbank-id']['#text']
    
    try:
        for interaction in obj['drug-interactions']['drug-interaction']:
            dst_id = interaction['drugbank-id']
            intr_dicts.append(
                  {
                    'source':drug_id,
                    'dst':dst_id,
                    'desc':interaction['description']
                  }
            )
    except: 
        pass
    

  0%|          | 0/15235 [00:00<?, ?it/s]

In [27]:
pd.DataFrame(intr_dicts).to_csv("temp/drug-interactions.tsv", sep="\t")

## Process Indications

Use disease names from ICD 11 table. Then use Fuzzy search

In [4]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm

In [5]:
# search_terms = open("../data/diseases/DisGeNET/tables/diseaseFiltered.csv")
# search_terms = search_terms.read()
# search_terms = search_terms.split("\n")
search_terms_df = pd.read_csv("../data/diseases/DisGeNET/tables/diseaseFiltered.csv", sep="\t")
search_terms_df

,diseaseId,diseaseName,type,MeSH_tree_number,MeSH_heading
0,C0024286,Lymphogranuloma Venereum,disease,C01,Infections
1,C0001249,Actinobacillus Infections,disease,C01,Infections
2,C0339916,Viral upper respiratory tract infection,group,C01,Infections
3,C1274320,Recurrent herpes simplex infection of eye,disease,C01,Infections
4,C0001261,Actinomycosis,group,C01,Infections
...,...,...,...,...,...
38319,C2700439,MAJOR AFFECTIVE DISORDER 8,disease,F03,Mental Disorders
38320,C0001306,Acute alcoholic liver disease,disease,F03,Mental Disorders
38321,C0236816,"Stress Disorders, Traumatic, Acute",group,F03,Mental Disorders
38322,C0221745,Depression suicidal,disease,F03,Mental Disorders


In [8]:
drug_indications = []

for obj in dat:
    # drugbank-id
    try:
        k,v = obj['drugbank-id'][0].items()
        id = v[1]
    except:
        id = obj['drugbank-id']['#text']

    indication = obj['indication']
    drug_indications.append(
        {
        'drugbank-id':id,
        'indication':indication
        }
    )

indications_df = pd.DataFrame(drug_indications)
#indications_df.to_clipboard()

indications_df

,drugbank-id,indication
0,DB00001,Lepirudin is indicated for anticoagulation in ...
1,DB00002,Cetuximab indicated for the treatment of local...
2,DB00003,Used as adjunct therapy in the treatment of cy...
3,DB00004,For treatment of cutaneous T-cell lymphoma
4,DB00005,Etanercept is indicated for the treatment of m...
...,...,...
15230,DB17382,None
15231,DB17383,None
15232,DB17384,None
15233,DB17385,None


In [9]:
#run on different system
drug_diseases = []

threshold = 70

for idx, row in tqdm(indications_df.iterrows(), total=len(indications_df)):
    for s_idx, s_row in search_terms_df.iterrows():
        search_term = s_row["diseaseName"]
        value = fuzz.token_set_ratio(search_term, row['indication'])
        if value > threshold:
            drug_diseases.append(
                {
                'drugbank-id':row['drugbank-id'],
                'indication':row['indication'],
                'disease-title':search_term,
                'disease-type':s_row['type'],
                'MeSH-tree':s_row['MeSH_tree_number'],
                'disease-category':s_row['MeSH_heading'],
                'match-score':value
                }  
            )

100%|██████████| 15235/15235 [11:48:58<00:00,  2.79s/it]       


In [10]:
pd.DataFrame(drug_diseases).to_excel("./temp/drug_diseases.xlsx")